# TUGAS BESAR KEAMANAN INFORMASI DAN JARINGAN

In [58]:
from PIL import Image

In [ ]:
rotor_1 = ["BDFHJLCPRTXVZNYEIWGAKMUSQO","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
rotor_2 = ["AJDKSIRUXBLHWTMCQGZNPYFVOE","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
rotor_3 = ["EKMFLGDQVZNTOWYHXUSPAIBRCJ","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]

reflector = ["YRUHQSLDPXNGOKMIEBFZCWVJAT","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
pludge_board = [["A","X"],["C","D"]]

In [59]:

rotor_1 = ["BDFHJLCPRTXVZNYEIWGAKMUSQO","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
rotor_2 = ["AJDKSIRUXBLHWTMCQGZNPYFVOE","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
rotor_3 = ["EKMFLGDQVZNTOWYHXUSPAIBRCJ","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]

reflector = ["YRUHQSLDPXNGOKMIEBFZCWVJAT","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]

rotor_1Position = 0
rotor_2Position = 0
rotor_3Position = 0

alfabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def plugboard(character):
    pludge_board = [["A","X"],["C","D"]]
    for i in pludge_board:
        if character in i[0]:
            return i[1]
        elif character in i[1]:
            return i[0]
    return character

def rotor_encription(character):
    global rotor_1,rotor_2,rotor_3,rotor_1Position,rotor_2Position,rotor_3Position,reflector
    
    # ROTOR 1
    rotor_1 = [rotor_1[0][rotor_1Position:] + rotor_1[0][:rotor_1Position], rotor_1[1][rotor_1Position:] + rotor_1[1][:rotor_1Position]]
    rotor1 = rotor_1[1].index(rotor_1[0][alfabet.index(character)])
    rotor_1Position += 1
    # ROTOR 2
    if rotor_1Position > len(alfabet):
        rotor_1Position = 0
        rotor_2 = [rotor_2[0][rotor_2Position:] + rotor_2[0][:rotor_2Position], rotor_2[1][rotor_2Position:] + rotor_2[1][:rotor_2Position]]
        rotor_2Position += 1
    rotor2 = rotor_2[1].index(rotor_2[0][rotor1])

    # ROTOR 3
    if rotor_2Position > len(alfabet):
        rotor_2Position = 0
        rotor_3 = [rotor_3[0][rotor_3Position:] + rotor_3[0][:rotor_3Position], rotor_3[1][rotor_3Position:] + rotor_3[1][:rotor_3Position]]
    rotor3 = rotor_3[1].index(rotor_3[0][rotor2])  
    
    if rotor_3Position > len(alfabet):
        rotor_3Position = 0
    # REFLECTOR
    reflector_1 = reflector[1].index(reflector[0][rotor3])
    reflector_1 = rotor_3[0].index(rotor_3[1][reflector_1])
    reflector_1 = rotor_2[0].index(rotor_2[1][reflector_1])
    return  alfabet[rotor_1[0].index(rotor_1[0][rotor_1[0].index(rotor_1[1][reflector_1])])]
  
    
def encription_process(character):
    global rotor_1,rotor_2,rotor_3,rotor_1Position,rotor_2Position,rotor_3Position,reflector
    result =  plugboard(rotor_encription(plugboard(character)))
    
    rotor_1 = ["BDFHJLCPRTXVZNYEIWGAKMUSQO","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
    rotor_2 = ["AJDKSIRUXBLHWTMCQGZNPYFVOE","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
    rotor_3 = ["EKMFLGDQVZNTOWYHXUSPAIBRCJ","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
    reflector = ["YRUHQSLDPXNGOKMIEBFZCWVJAT","ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
    return result

In [60]:
def encode_message(image_path, message, output_path):
    # Open the image
    img = Image.open(image_path)
    print(f'1. READ IMAGE DATA :\n{img}\n\n')

    # Convert the message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    print(f'2. CONVERT MESSAGE :\n\nOriginal Message\n{message}\n\nBinary Message\n{binary_message} with lenght of {len(binary_message)}\n\n')

    # Ensure the image can contain the message
    if len(binary_message) > img.size[0] * img.size[1] * 3:
        raise ValueError("Message is too long for the image")

    # Encode the message in the image
    data_index = 0
    img_data = list(img.getdata())
    new_img_data = []
    print(f"3. ENCODE MESSAGE TO THE IMAGE :\n\nFirst Step:\n{img_data[:5]}\n\nSecond Step\n")
    for pixel in img_data:
        if data_index < len(binary_message):
            # Convert the pixel values to binary
            pixel = list(pixel)
            print(f'{data_index}. Merge Logic')
            for i in range(3):  # RGB channels
                if data_index < len(binary_message):
                    print(f'{pixel[i]} ==> {pixel[i] & ~1 | int(binary_message[data_index], 2)} = ', end="")
                    pixel[i] = pixel[i] & ~1 | int(binary_message[data_index], 2)
                    data_index += 1
                    print(f'{pixel[i]}')
                
            pixel = tuple(pixel)
            print(f'Result = {pixel}\n')
        new_img_data.append(pixel)
        
    # Create a new image with the encoded data
    new_img = Image.new(img.mode, img.size)
    new_img.putdata(new_img_data)
    print(f'\n\n4. FINAL RESULT :\n\n{new_img}')
    # Save the new image
    new_img.save(output_path)

In [61]:
def decode_message(image_path):
    img = Image.open(image_path)
    img_data = list(img.getdata())

    binary_message = ''
    for pixel in img_data:
        for i in range(3):
            binary_message += str(pixel[i] & 1)

    # Convert binary message to ASCII
    message = ''.join([chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8)])
    return message

In [85]:
# Example usage:
# Encode message in an image
rotor_1Position = int(input()) % 26
rotor_2Position = int(input()) % 26 
rotor_3Position = int(input()) % 26

pesan = input("masukkan pesan : ").upper()
encription = ""

for i in pesan:
    encription += encription_process(i)
print(f'encryption text = {encription}')
    
encode_message('sticker7.jpg', encription, 'encoded_image.png')

# Decode message from the encoded image


encryption text = KPGNDPOCCXZ
1. READ IMAGE DATA :
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=403x199 at 0x1CF1CE66308>


2. CONVERT MESSAGE :

Original Message
KPGNDPOCCXZ

Binary Message
0100101101010000010001110100111001000100010100000100111101000011010000110101100001011010 with lenght of 88


3. ENCODE MESSAGE TO THE IMAGE :

First Step:
[(34, 34, 34), (34, 34, 34), (35, 35, 35), (35, 35, 35), (36, 36, 36)]

Second Step

0. Merge Logic
34 ==> 34 = 34
34 ==> 35 = 35
34 ==> 34 = 34
Result = (34, 35, 34)

3. Merge Logic
34 ==> 34 = 34
34 ==> 35 = 35
34 ==> 34 = 34
Result = (34, 35, 34)

6. Merge Logic
35 ==> 35 = 35
35 ==> 35 = 35
35 ==> 34 = 34
Result = (35, 35, 34)

9. Merge Logic
35 ==> 35 = 35
35 ==> 34 = 34
35 ==> 35 = 35
Result = (35, 34, 35)

12. Merge Logic
36 ==> 36 = 36
36 ==> 36 = 36
36 ==> 36 = 36
Result = (36, 36, 36)

15. Merge Logic
37 ==> 36 = 36
37 ==> 36 = 36
37 ==> 37 = 37
Result = (36, 36, 37)

18. Merge Logic
37 ==> 36 = 36
37 ==> 36 = 36
37 ==> 36 = 3

In [86]:
import re

def filter_huruf(s):
    return re.sub(r'[^A-Z]', '', s)

decoded_message = decode_message('encoded_image.png')
print("Decoded Message in Image:", decoded_message)

decoded_message = filter_huruf(decoded_message)
print(decoded_message)

rotor_1Position = int(input()) % 26
rotor_2Position = int(input()) % 26 
rotor_3Position = int(input()) % 26

# pesan = input("masukkan pesan : ").upper()
pesan = decoded_message
description = ""

for i in pesan:
    description += encription_process(i)

print("Decoded Enigma Message:", description)


Decoded Message in Image: KPGNDPOCCXZÇÇà~  À~8  8àp?~ãÿÿñøøã ð àÇþü~ ÿñÇ ~8  qÿÿÀãÿÿÿÀø8à~ àp?8Àþ?à~ãÿÇÿ p?ü~?ÿøà?à?ãð ?ãð8ãÿÿ À 8Çÿþ p ÿ?þ?ñÇüÿñÀþ8~øÿ8qøÿÿð8üÀàqÀÿÀÀãÿüøàÀàÀ ~  ÿàÿü  qÿÿÀÀøÿñø ?p8? À qø    ðÇð þ8ÿ?ÿð üp8ãñÀã ø p?ã8ÿÀüÀøþ?ãð ãþà þ ÿÿãð üÇÿñÀÿÀüø ÿãÿÿÿÇàÀÿð8?ÿÿpÿ8 Ç~àÇàp8ÀüÇþ8ÿ Àÿ?þp88ñÀþ8ÿñÀþ ÿÿÿÇãø~   ÿÀ~?ÿðÿà? ÿÿÿüqÇü~ü? À øþ?üøàÀÿ8Àã8ÿãð?ãÀüp ÿ?ÿÿÿÿ8  ?ãÇàp?ãÿÇðãñÇàø?üp8àøð?8 8ð8  Çàÿüp8à  þ8ÿÿÇÿ?p8ñÇñÿÇ  ÿÿð?ÿÿÿÿüÿÿÿÿÿÿÿÿ? ÿÿÿÿÿÿøÿÿÿñÿÿÿÿÿÿøÿÿøð?ÿÿÿÿÿÀ ÿÿÿÇÿÿøÿÿÿÿÿÿþ?ÿÿÿÇÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿð?üÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿüÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿø ÿüøþ ÿàøÿ8ãÀ  ÿÀ  8ø  üøü?ÿÿà ü ?ÿÿÿ 8  ÿþÿ À~ÿàÿø    ãñÀ üÇÇãð8 ÿàÀàp?ü üøÿÿÿÇàÿÿÀðÿÿøàqÇÿÿÿÇü ÿð ãþÿþ à ãþ?ãÀàÀÿð ñÿøÀpð ü8ãÇð8ü~ ÿñÿÿÇð